## Question 2

Create a dataframe containing data toronto neghborhood

In [1]:
import requests
import pandas as pd
import re

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml')
table=soup.find('table')

column_names = ['PostalCode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

# Read data from table and copy in dataframe

for tr in table.find_all('tr'):
    for td in tr.find_all('td'):
        data = td.text.strip('\n')
        postcode = data[0:3]
        data = data.replace(postcode,'')
        temp = re.findall('[\w+ \'\.]+',data)
        borough =temp[0]
        for neighborhood in temp[1:len(temp)]:
            neighborhood=neighborhood.lstrip(" ").rstrip(" ")
            toronto = toronto.append({'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
        
toronto['Borough']=toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto = toronto[toronto['Borough']!='Not assigned']

for idx,row in toronto.iterrows():
    if row['Neighborhood'] == "":
        toronto.loc[idx]['Neighborhood']=row['Borough']

toronto_neighbors = toronto.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toronto_neighbors

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Read data from csv file from an url 

In [ ]:
import csv

toronto_geocsv = 'https://cocl.us/Geospatial_data'
geocsv_data = pd.read_csv(toronto_geocsv)#.set_index("Postal Code")
geocsv_data = geocsv_data.rename(columns={'Postal Code':'PostalCode'})
geocsv_data.head()

Merge two dataframes to get latitude and longitude of a location with repsect to a postal code

In [ ]:
merge_toronto_neighbors_geocsv_data = pd.merge(toronto_neighbors,geocsv_data,on='PostalCode')
merge_toronto_neighbors_geocsv_data

Save output of this assignment in csv file to use in question 3

In [ ]:
merge_toronto_neighbors_geocsv_data.to_csv('toronto_neighborhood.csv')

In [ ]:
print(merge_toronto_neighbors_geocsv_data.shape)